In [1]:
import os
import glob
import geemap
import ee, datetime
from IPython.display import Image
import pandas as pd
import geopandas as gpd
from pylab import *
import seaborn as sns
import folium
import matplotlib.pyplot as plt
%matplotlib inline
import altair as alt

In [2]:
# Getting tokens and activate EE
ee.Authenticate()

# Start up this session
ee.Initialize()

Enter verification code: 4/1AWtgzh4V59Lus8JPpL9HuO5OIVi5unJaUYdVIYNkQOUZAaPhbIaBJMxL_Bk

Successfully saved authorization token.
